In [1]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import math
import glob, os


In [2]:
class wall:
    def __init__(self, cell_index, cell_center_x, cell_center_y, wall_index):
        self.translation_x = cell_center_x
        self.translation_y = cell_center_y
        self.translation_z = 25
        if wall_index == 0:
            self.direction = 'North'
            self.translation_y += 90
        elif wall_index == 1:
            self.direction = 'East'
            self.translation_x += 90
        elif wall_index == 2:
            self.direction = 'South'
            self.translation_y -= 90
        elif wall_index == 3:
            self.direction = 'West'
            self.translation_x -= 90
        self.name = 'Cell ' + str(cell_index) + ' ' + self.direction + ' Wall' 
class cell:
    def __init__(self, index=0, wall_code=0, wall_config=[0,0,0,0]):
        self.index = index
        self.wall_code = wall_code
        self.wall_config = wall_config
        # Sets the row and column index relative to a 16x16 maze
        self.cell_row, self.cell_col = divmod(index,16)
        # Sets the row and column index relative of the cell to the North and West
        # used to avoid duplicating walls
        self.cell_to_north_row = self.cell_row - 1
        self.cell_to_north_col = self.cell_col
        self.cell_to_west_row = self.cell_row
        self.cell_to_west_col = self.cell_col - 1
        # Sets the (x,y) center of the cell relative the the maze (in mm)
        self.center_x = ((90) + (self.cell_col*180))
        self.center_y = ((2880 - 90) - (self.cell_row*180))
        self.walls = []
        for i in range(4):
            if wall_config[i] == 1:
                self.walls.append(wall(self.index, self.center_x, self.center_y,i))
        
class maze:
    def __init__(self, cell_codes, Size = 16):
        wall_dict = {
            0 : [0,0,0,0],
            1 : [1,0,0,0],
            2 : [0,1,0,0],
            4 : [0,0,1,0],
            8 : [0,0,0,1],
            3 : [1,1,0,0],
            5 : [1,0,1,0],
            9 : [1,0,0,1],
            6 : [0,1,1,0],
            10 : [0,1,0,1],
            12 : [0,0,1,1],
            7 : [1,1,1,0],
            11 : [1,1,0,1],
            13 : [1,0,1,1],
            14 : [0,1,1,1],
            15 : [1,1,1,1],

        }
        self.maze = np.full((Size, Size), cell())
        self.maze_list = []
        indexer = 0
        for cell_code in cell_codes:
            r,c = divmod(indexer,Size)
            cell_and_walls = cell(indexer,cell_code,wall_dict[cell_code])
            self.maze[r][c] = cell_and_walls
            self.maze_list.append(cell_and_walls)
            indexer+=1

In [3]:
def get_maze_data(file):
    dtype = np.dtype('B') 
    with open(file, "rb") as f:
        maze_data = np.ravel(np.rot90(np.fromfile(f,dtype).reshape((16,16))))
    return maze_data

def get_wall_text(wall):
    # Sets the text to add the wall to the webots world file
    if wall.direction == 'North' or wall.direction == 'South':
        start_line = 'MazeWallNorthSouth {\n'
    else:
        start_line = 'MazeWallEastWest {\n'
    translation_line =  '\t' + 'translation ' + str(wall.translation_x/1000) + ' ' + str(wall.translation_y/1000) + ' ' + str(wall.translation_z/1000) + '\n'
    name_line = '\t' + 'name "' + wall.name + '"\n'
    end_line = '}\n'
    return start_line + translation_line + name_line + end_line



In [4]:
dtype = np.dtype('B')
mazes = []
for file in glob.glob("mazefiles/binary/*.maz"):
    # print(file)
    maze_data = get_maze_data(file)
    # print(maze_data)
    mazes.append(maze(maze_data))



In [5]:
# Write the world file for webots
# Copy template to new file
counter = 1
for m in mazes:
    with open('template.wbt','r') as template, open('MicroMouse/worlds/Maze'+str(counter)+'.wbt','w') as mazefile:
        # read content from first file
        for line in template:
            # append content to second file
            mazefile.write(line)
        for r in range(len(m.maze)):
            for c in range(len(m.maze[r])):
                for wall in m.maze[r,c].walls:
                    if r == 0:
                        mazefile.write(get_wall_text(wall))
                    elif c == 0 and wall.direction != 'North':
                        mazefile.write(get_wall_text(wall))
                    else:
                        if wall.direction == 'East' or wall.direction == 'South':
                            mazefile.write(get_wall_text(wall))
                        
    counter += 1
